In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

In [2]:
! pip install git+https://github.com/huggingface/datasets.git

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-d9w6dtzd
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-d9w6dtzd
     |████████████████████████████████| 245kB 4.2MB/s 
     |████████████████████████████████| 112kB 7.6MB/s 
  Created wheel for datasets: filename=datasets-1.6.0.dev0-cp37-none-any.whl size=199488 sha256=cdefd8a932df304491b5e448cbf759476da96d3ca20e9f87d87affcc1940a8f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-u2b466hc/wheels/3e/af/ff/d1cdb5d0f9cff6eba2042a16b477ada497e23f1a3b6950b928
Successfully built datasets


In [3]:
model_checkpoint = "bert-base-uncased"
batch_size = 16

In [4]:
from datasets import load_dataset, load_metric

In [5]:
datasets = load_dataset("swag","regular")

Dataset swag downloaded and prepared to /root/.cache/huggingface/datasets/swag/regular/0.0.0/32c6a4e9a3c0880c9ace1df43c617cf89b7816037ebf2feabd4595ec4a02f74e. Subsequent calls will reuse this data.


In [6]:
datasets

DatasetDict({
    train: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 73546
    })
    validation: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20006
    })
    test: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20005
    })
})

In [7]:
datasets["train"][0]

{'ending0': 'passes by walking down the street playing their instruments.',
 'ending1': 'has heard approaching them.',
 'ending2': "arrives and they're outside dancing and asleep.",
 'ending3': 'turns the lead singer watches the performance.',
 'fold-ind': '3416',
 'gold-source': 'gold',
 'label': 0,
 'sent1': 'Members of the procession walk down the street holding small horn brass instruments.',
 'sent2': 'A drum line',
 'startphrase': 'Members of the procession walk down the street holding small horn brass instruments. A drum line',
 'video-id': 'anetv_jkn6uvmqwh4'}

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

In [9]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset)
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(datasets["train"])

,ending0,ending1,ending2,ending3,fold-ind,gold-source,label,sent1,sent2,startphrase,video-id
0,stand around an outdoor counter covered in snow.,are dancing in a car.,help it out of the car.,ski inside the warehouse.,662,gen,0,A car drives away from the camera.,Two people,A car drives away from the camera. Two people,anetv_9V7cMp_w1_0
1,"gazes at the ground, then lowers it to the beat - up mirror.",turns on a cement surface and shuts the door.,then walks in the direction of the actual snake and pauses.,"stands still, horrified.",14258,gold,3,Someone turns a bottle around from which someone has taken the powder for the capsules.,Someone,Someone turns a bottle around from which someone has taken the powder for the capsules. Someone,lsmdc0014_Ist_das_Leben_nicht_schoen-54247
2,is rubbing with a crash to the right side of the table.,"reaches for the falcon but misses, then falls into a building.",opens it to reveal the massive diamond.,"takes it out, examines the drawers and the iron.",16468,gold,1,The handlebars break on a crossbeam and someone swings on to a paper lantern.,He,The handlebars break on a crossbeam and someone swings on to a paper lantern. He,lsmdc3066_THE_ADVENTURES_OF_TINTIN-31920
3,reach the bottom of the hill.,continue to get through the water and are standing on a pool.,finish by tying their rake down and laying them floors.,crowd around on the rough stands.,19094,gold,0,Eventually some of them go down the hill.,They,Eventually some of them go down the hill. They,anetv_I-1UKECfQko
4,knocks off the camera.,presses the elevator button into the other street.,feels around for the door of the limo.,opens the box at a utility clip.,8846,gold,2,Someone escorts a petite redhead down the building's front steps toward his brother and the limo.,Someone,Someone escorts a petite redhead down the building's front steps toward his brother and the limo. Someone,lsmdc3011_BLIND_DATING-750
5,goes on with the phone.,"quickly finds a piece of machinery, and studies it through his open window.",ends the sight and cranks the key into the lock.,"looks off in another direction, slightly behind the office, and sees.",14759,gold,3,"Someone goes to the desk, rings a small pushbell.",She,"Someone goes to the desk, rings a small pushbell. She",lsmdc0038_Psycho-67224
6,is shown again in slow motion.,is shown several more times as well as several others watching on the sides.,is shown as well as a ball being played in a goal.,is shown again in slow motion while practicing cheers.,17394,gold,0,A small group of people are seen running down a field when one scores a goal.,The same shot,A small group of people are seen running down a field when one scores a goal. The same shot,anetv_8Q-P5KEvXN0
7,are dancing and holding a link to them.,"exchange glances, then head inside, followed by people.","run into the woods, swoop in the trees and hold others a few yards behind them.",are walking in the opposite direction as someone banks out.,3630,gold,1,Someone makes to follow them.,People,Someone makes to follow them. People,lsmdc1044_Pride_And_Prejudice_Disk_Two-89871
8,is then seen looking at the camera while looking back to the camera and showing a thumbs up.,continues spinning that around and ends by taking a sip of the cut.,continues brushing and blowing a bit of water while still looking off and picks up a piece of snow.,throws the ball around while others watch on the left.,1949,gen,3,A man is seen standing at the end of a dock and looking back to people partying.,The man,A man is seen standing at the end of a dock and looking back to people partying. The man,anetv_XJ-knBky6S0
9,plunges down a steep river toward someone.,makes the crowded sidewalk adorned with lights go out.,passes under a highway where several tanks sit burning.,walks up to someone's view.,6034,gold,2,She gives him a pat on the shoulder and leaves.,Now the bus,She gives him a pat on the shoulder and leaves. Now the bus,lsmdc3009_BATTLE_LOS_ANGELES-317


In [11]:
def show_one(example):
    print(f"Context: {example['sent1']}")
    print(f"  A - {example['sent2']} {example['ending0']}")
    print(f"  B - {example['sent2']} {example['ending1']}")
    print(f"  C - {example['sent2']} {example['ending2']}")
    print(f"  D - {example['sent2']} {example['ending3']}")
    print(f"\nGround truth: option {['A', 'B', 'C', 'D'][example['label']]}")

In [12]:
show_one(datasets["train"][0])

Context: Members of the procession walk down the street holding small horn brass instruments.
  A - A drum line passes by walking down the street playing their instruments.
  B - A drum line has heard approaching them.
  C - A drum line arrives and they're outside dancing and asleep.
  D - A drum line turns the lead singer watches the performance.

Ground truth: option A


In [13]:
show_one(datasets["train"][15])

Context: Now it's someone's turn to rain blades on his opponent.
  A - Someone pats his shoulder and spins wildly.
  B - Someone lunges forward through the window.
  C - Someone falls to the ground.
  D - Someone rolls up his fast run from the water and tosses in the sky.

Ground truth: option C


In [16]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [17]:
ending_names = ["ending0", "ending1", "ending2", "ending3"]

def preprocess_function(examples):
    first_sentences = [[context] * 4 for context in examples["sent1"]]
    question_headers = examples["sent2"]
    second_sentences = [[f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

In [18]:
examples = datasets["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 4 [30, 25, 30, 28]


In [19]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

In [20]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

['[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession are playing ping pong and celebrating one left each in quick. [SEP]',
 '[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession wait slowly towards the cadets. [SEP]',
 '[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession makes a square call and ends by jumping down into snowy streets where fans begin to take their positions. [SEP]',
 '[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession play and go back and forth hitting the drums while the audience claps for them. [SEP]']

In [21]:
show_one(datasets["train"][3])

Context: A drum line passes by walking down the street playing their instruments.
  A - Members of the procession are playing ping pong and celebrating one left each in quick.
  B - Members of the procession wait slowly towards the cadets.
  C - Members of the procession makes a square call and ends by jumping down into snowy streets where fans begin to take their positions.
  D - Members of the procession play and go back and forth hitting the drums while the audience claps for them.

Ground truth: option D


In [22]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

In [23]:
args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [24]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

In [25]:
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
      
    def __call__(self, features):
         label_name = "label" if "label" in features[0].keys() else "labels"
         labels = [feature.pop(label_name) for feature in features]
         batch_size = len(features)
         num_choices = len(features[0]["input_ids"])
         flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
         flattened_features = sum(flattened_features, [])
         batch = self.tokenizer.pad(flattened_features,padding=self.padding,max_length=self.max_length,pad_to_multiple_of=self.pad_to_multiple_of,return_tensors="pt")
         batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
         batch["labels"] = torch.tensor(labels, dtype=torch.int64)
         return batch

In [26]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch =  DataCollatorForMultipleChoice(tokenizer)(features)

In [27]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [28]:
args = TrainingArguments(evaluation_strategy = "epoch",learning_rate=5e-5,per_device_train_batch_size=batch_size,
per_device_eval_batch_size=batch_size,num_train_epochs=3,weight_decay=0.01,output_dir='/content/output_dir')

In [29]:
trainer = Trainer(model,args,train_dataset=encoded_datasets["train"],eval_dataset=encoded_datasets["validation"],tokenizer=tokenizer,
data_collator=DataCollatorForMultipleChoice(tokenizer),compute_metrics=compute_metrics,)

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored